# (Required Imports)

In [ ]:
# TODO(deflaux): remove this cell after gcr.io/broad-ml4cvd/deeplearning:tf2-latest-gpu has this preinstalled.
from ml4h.runtime_data_defines import determine_runtime
from ml4h.runtime_data_defines import Runtime

if Runtime.ML4H_VM == determine_runtime():
  !pip3 install --user ipycanvas==0.7.0 ipyannotations==0.2.1
  !jupyter nbextension install --user --py ipycanvas
  !jupyter nbextension enable --user --py ipycanvas
  # Be sure to restart the kernel if pip installs anything.
  # Also, shift-reload the browser page after the notebook extension installation.

# Restart kernel here !!!

In [1]:
from ipyannotations import BoxAnnotator, PointAnnotator, PolygonAnnotator
from ipyannotations.images.annotator import Annotator
from ml4h.visualization_tools.annotation_storage import DataFrameAnnotationStorage
from ml4h.visualization_tools.annotations import T1MAPPointAnnotator
from ml4h.visualization_tools.batch_image_annotations import BatchImageAnnotator
import pandas as pd
import tensorflow as tf

pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  if __name__ == '__main__':


In [2]:
%%javascript
// Display cell outputs to full height (no vertical scroll bar)
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

# Define the batch of samples to annotate

In [3]:
USER = 'pdiachil'

png_df = pd.read_csv(f'png_split_{USER}.csv')
png_df

,sample_id,tmap_name,instance_number,folder
0,4754055,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
1,3677123,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
2,2342832,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
3,4775078,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
4,1176560,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
...,...,...,...,...
11365,2304541,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
11366,5535387,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
11367,1273968,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
11368,4911555,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output


### Use code below to eventually exclude images that you may have already annotated

In [4]:
# List the dataframes already annotated by the user
import glob
annotated_by_user = glob.glob(f'goodbad_t1map_inference_{USER}*')
annotated_by_user

['goodbad_t1map_inference_pdiachil_05_22_21_14_11_41.csv',
 'goodbad_t1map_inference_pdiachil_05_22_21_14_15_50.csv',
 'goodbad_t1map_inference_pdiachil_05_22_21_14_06_01.csv']

In [5]:
# If you want to exclude any of the images from the CSVs above, please add the filenames (separatedy by commas) 
# to the 'to_exclude' list
to_exclude = [
    'goodbad_t1map_inference_pdiachil_05_22_21_14_15_50.csv',
    'goodbad_t1map_inference_pdiachil_05_22_21_14_06_01.csv'
]

In [6]:
# Remove images in to_exclude
for to_exclude_csv in to_exclude:
    to_exclude_df = pd.read_csv(to_exclude_csv)
    png_df = pd.merge(png_df, to_exclude_df[['sample_id', 'instance_number']], indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1).reset_index(drop=True)
png_df

,sample_id,tmap_name,instance_number,folder
0,3424227,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
1,2814467,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
2,3971630,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
3,2356531,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
4,4182789,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
...,...,...,...,...
11339,2304541,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
11340,5535387,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
11341,1273968,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
11342,4911555,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output


# Annotate the batch! 

In [7]:
# Note: a zoom level of 1.0 displays the tensor as-is. For higher zoom levels, this code currently
# use the PIL library to scale the image.
import datetime

output_filename = f'goodbad_t1map_inference_{USER}_{datetime.datetime.now().strftime("%m_%d_%y_%H_%M_%S")}.csv'

annotator = BatchImageAnnotator(samples=png_df,
                                zoom=1.5,
                                annotation_categories=['good', 'bad', 'review'],
                                annotation_storage=DataFrameAnnotationStorage(output_filename),
                                annotator=T1MAPPointAnnotator,
                                user=USER
                               )
annotator.annotate_images()

# View the stored annotations 

In [ ]:
annotator.view_recent_submissions(count=100)

# Provenance

In [ ]:
import datetime
print(datetime.datetime.now())

In [ ]:
%%bash
pip3 freeze

In [ ]:
annotator.annotation_widget.data

Questions about these particular notebooks? Join the discussion https://github.com/broadinstitute/ml4h/discussions.